# Tests

In [1]:
import os
import sys
import json
import matplotlib.image as mpimg
from matplotlib import pyplot as plt
from termcolor import colored
import warnings
warnings.filterwarnings('ignore')

# change this property
NOMEROFF_NET_DIR = "/var/www/nomeroff-net/"
MASK_RCNN_DIR = "/var/www/Mask_RCNN/"

MASK_RCNN_LOG_DIR = os.path.join(NOMEROFF_NET_DIR, "logs/")
MASK_RCNN_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, "models/mask_rcnn_numberplate_0700.h5")
REGION_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, "models/imagenet_vgg16_np_region_2019_1_18.h5")
FILL_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, "models/imagenet_vgg16_np_filled_2019_1_15.h5")
OCR_NP_TEXT = "/mnt/datasets/anpr_ocr/models/ocr_np_text.h5"

sys.path.append(NOMEROFF_NET_DIR)

In [2]:
from NomeroffNet import  filters, RectDetector, TextDetector, RegionDetector, Detector, textPostprocessing, FillDetector

nnet = Detector(MASK_RCNN_DIR, MASK_RCNN_LOG_DIR)
nnet.loadModel(MASK_RCNN_MODEL_PATH)

rectDetector = RectDetector()
textDetector = TextDetector()
regionDetector = RegionDetector()
regionDetector.load(REGION_MODEL_PATH)
fillDetector = FillDetector()
fillDetector.load(FILL_MODEL_PATH)

ukrTextDetector = TextDetector("ocr_ukr")
ukrTextDetector.load(OCR_NP_TEXT)

Using TensorFlow backend.


Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


In [3]:
def test(dirName, fname, y, verbose=0):
    img_path = os.path.join(dirName, fname)
    if verbose==1:
        print(colored(f"__________ \t\t {img_path} \t\t __________", "blue"))
    img = mpimg.imread(img_path)
    NP = nnet.detect([img])
    if verbose==1:
        print(colored("MASK_RCNN RESULTS:", "blue"))
        print(NP)
    cv_img_masks = filters.cv_img_mask(NP) 
    nGood = 0
    nBad = 0
    res = []
    for img_mask in cv_img_masks:
        #mpimg.imsave("masks/mask.png", img_mask)
        points = rectDetector.detect(img_mask, fixRectangleAngle=1, outboundWidthOffset=3)
        #print(rectDetector.checkIfIsSquare(points)) # buff
        if verbose==1:
            print(colored("RECT DETECTOR RESULTS:", "blue"))
            print(points)
        zone = rectDetector.get_cv_zones(img, points)
        #print(fillDetector.predict(zone))
        if verbose==1:
            plt.axis("off")
            plt.imshow(zone)
            plt.show()
        regionId = regionDetector.predict(zone)
        regionName = regionDetector.getLabels(regionId)
        if verbose==1:
            print(colored("REGION NAME:", "blue"))
            print(regionName)
            
        if regionId in (1, 2):
            print("Ukr text detctor")
            text = ukrTextDetector.predict(zone)
        else:
            text = textDetector.detect(zone)
        if verbose==1:
            print(colored("SOURCE TEXT:", "blue"))
            print(text)
        text = textPostprocessing(text, regionName)
        if verbose==1:
            print(colored("POSTPROCESSED TEXT:", "blue"))
            print(text)
        res.append(text)
    for yText in y:
        if yText in res:
            print(colored(f"OK: TEXT:{yText} \t RESULTS:{res} \t\t in PATH:{img_path}", 'green'))
            nGood += 1
        else:
            print(colored(f"NOT OK: TEXT:{yText} \t RESULTS:{res} \t\t in PATH:{img_path} ", 'red'))
            nBad += 1
    return nGood, nBad

In [4]:
dirName = "images/"
testData = {
    "0.jpeg": ["AI5255EI"],
    "1.jpeg": ["HH7777CC"],
    "2.jpeg": ["AT1515CK"],
    "3.jpeg": ["BX0578CE"],
    "4.jpeg": ["AC4249CB"],
    "5.jpeg": ["BC3496HC"],
    "6.jpeg": ["BC3496HC"],
    "7.jpeg": ["AO1306CH"],
    "8.jpeg": ["AE1077CO"],
    "9.jpeg": ["AB3391AK"],
    "10.jpeg": ["BE7425CB"],
    "11.jpeg": ["BE7425CB"],
    "12.jpeg": ["AB0608EA"],
    "13.jpeg": ["AB0680EA"],
    "14.jpeg": ["BM1930BM"],
    "15.jpeg": ["AI1382HB"],
    "16.jpeg": ["AB7333BH"],
    "17.jpeg": ["AB7642CT"],
    "18.jpeg": ["AC4921CB"],
    "19.jpeg": ["BC9911BK"],
    "20.jpeg": ["BC7007AK"],
    "21.jpeg": ["AB5649CI"],
    "22.jpeg": ["AX2756EK"],
    "23.jpeg": ["AA7564MX"],
    "24.jpeg": ["AM5696CK"],
    "25.jpeg": ["AM5696CK"],
}

gGood = 0
gBad = 0
i = 0
for fileName in testData.keys():
    print(f"step {i}")
    nGood, nBad = test(dirName, fileName, testData[fileName])
    gGood += nGood
    gBad += nBad
    i += 1
total = gGood + gBad
print(f"TOTAL GOOD: {gGood/total}")
print(f"TOTAL BED: {gBad/total}")

step 0
Ukr text detctor
OK: TEXT:AI5255EI 	 RESULTS:['AI5255EI'] 		 in PATH:images/0.jpeg
step 1
Ukr text detctor
OK: TEXT:HH7777CC 	 RESULTS:['HH7777CC'] 		 in PATH:images/1.jpeg
step 2
Ukr text detctor
OK: TEXT:AT1515CK 	 RESULTS:['AT1515CK', 'NRP40881F'] 		 in PATH:images/2.jpeg
step 3
Ukr text detctor
OK: TEXT:BX0578CE 	 RESULTS:['BX0578CE'] 		 in PATH:images/3.jpeg
step 4
Ukr text detctor
OK: TEXT:AC4249CB 	 RESULTS:['AC4249CB'] 		 in PATH:images/4.jpeg
step 5
Ukr text detctor
OK: TEXT:BC3496HC 	 RESULTS:['BC3496HC'] 		 in PATH:images/5.jpeg
step 6
Ukr text detctor
OK: TEXT:BC3496HC 	 RESULTS:['BC3496HC'] 		 in PATH:images/6.jpeg
step 7
Ukr text detctor
OK: TEXT:AO1306CH 	 RESULTS:['AO1306CH'] 		 in PATH:images/7.jpeg
step 8
Ukr text detctor
OK: TEXT:AE1077CO 	 RESULTS:['AE1077CO'] 		 in PATH:images/8.jpeg
step 9
Ukr text detctor
OK: TEXT:AB3391AK 	 RESULTS:['AB3391AK'] 		 in PATH:images/9.jpeg
step 10
Ukr text detctor
OK: TEXT:BE7425CB 	 RESULTS:['BE7425CB'] 		 in PATH:images/10.